<a href="https://colab.research.google.com/github/colincockburn/CISC_473_project/blob/main/image_restoration_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell at start of each session - Change BRANCH name to your working branch.
# if pushing changes to this cell make sure to add them to both notebooks version of it.

import yaml, os
from google.colab import drive

# ==== Team Bootstrap (Colab) ====
# 1) Mount Drive (data only)
drive.mount('/content/drive')

# 2) define consts
REPO_URL  = "https://github.com/colincockburn/CISC_473_project.git"
REPO_DIR  = "/content/drive/MyDrive/projects/CISC_473_project/"
DATA_ROOT = "/content/drive/MyDrive/projects/CISC_473_project_data"

# 3) Clone repo if missing
import os, subprocess, sys
if not os.path.isdir(REPO_DIR):
    os.makedirs(os.path.dirname(REPO_DIR), exist_ok=True)
    print("Cloning repo to", REPO_DIR)
    !git clone "$REPO_URL" "$REPO_DIR"

# 4) Git identity (first time only; uncomment and change to your info)'
# dont push changes

%cd $REPO_DIR
!git config user.email "colincockburn64@gmail.com"
!git config user.name "colincockburn"

# 5) Sync with origin/main and create/update your feature branch
BRANCH = "cc-training"  # e.g., feature/cc-dataloader
if BRANCH:
  !git fetch origin
  !git checkout main
  !git pull origin main
  !git checkout -B "$BRANCH"
else:
  raise Exception("Please update BRANCH var to your desired working branch")

# 6) Sanity checks
import os
assert os.path.isdir(DATA_ROOT), f"Missing data at: {DATA_ROOT}"
print("OK: repo at", REPO_DIR)
print("OK: data at", DATA_ROOT)
!git status


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/projects/CISC_473_project
fatal: Unable to create '/content/drive/MyDrive/projects/CISC_473_project/.git/index.lock': File exists.

Another git process seems to be running in this repository, e.g.
an editor opened by 'git commit'. Please make sure all processes
are terminated then try again. If it still fails, a git process
may have crashed in this repository earlier:
remove the file manually to continue.
From https://github.com/colincockburn/CISC_473_project
 * branch            main       -> FETCH_HEAD
Already up to date.
fatal: Unable to create '/content/drive/MyDrive/projects/CISC_473_project/.git/index.lock': File exists.

Another git process seems to be running in this repository, e.g.
an editor opened by 'git commit'. Please make sure all processes
are terminated then try again. If it still fails, a git process
may have crashed i

In [2]:
# get configs
with open(f"{REPO_DIR}/configs/default.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [4]:
# run training
print("imports")
import sys
sys.path.append("/content/drive/MyDrive/projects/CISC_473_project/src")
from train import main
from train import print_thing
import torch

print("gpu availale: ", torch.cuda.is_available())

!mkdir -p ../checkpoints
data_root = cfg["paths"]["data_root"]
save_dir = cfg["paths"]["save_dir"]

sys.argv = [
    "train.py",
    "--data_root", data_root,
    "--save_dir", save_dir,
    "--epochs", "10",
    "--batch_size", "16",
    "--patch_size", "128",
    "--lr", "1e-3"
]

main()

imports
False
starting loop


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


NameError: name 'i' is not defined

In [ ]:
# monitor metrics

In [ ]:
# eval model

In [ ]:
# disply results